## Deliverable 2. Create a Customer Travel Destinations Map.

In [25]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [26]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bredasdorp,ZA,-34.5322,20.0403,73.27,60,0,17.27,clear sky
1,1,Mwinilunga,ZM,-11.7358,24.4293,64.92,94,94,3.51,overcast clouds
2,2,Port Elizabeth,ZA,-33.9180,25.5701,75.09,68,0,13.80,clear sky
3,3,Bethel,US,41.3712,-73.4140,19.85,81,100,5.01,haze
4,4,Victoria,HK,22.2855,114.1577,64.26,83,66,11.01,broken clouds


In [27]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = int(input('What is the minimum temperature you want in your trip?'))
max_temp = int(input('What is the maximum temperature you want in your trip?'))

What is the minimum temperature you want in your trip?75
What is the maximum temperature you want in your trip?90


In [28]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                 (city_data_df['Max Temp'] >= min_temp)]
preferred_cities.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Port Elizabeth,ZA,-33.9180,25.5701,75.09,68,0,13.80,clear sky
7,7,Bengkulu,ID,-3.8004,102.2655,76.48,90,100,3.83,overcast clouds
13,13,Neiafu,TO,-18.6500,-173.9833,82.49,71,50,13.71,scattered clouds
15,15,Hermanus,ZA,-34.4187,19.2345,83.89,62,0,20.94,clear sky
16,16,Salinopolis,BR,-0.6136,-47.3561,75.54,90,100,8.48,overcast clouds
20,20,Kavaratti,IN,10.5669,72.6420,81.19,68,6,3.60,clear sky
21,21,Faanui,PF,-16.4833,-151.7500,80.33,74,18,11.90,moderate rain
29,29,Ilebo,CD,-4.3167,20.5833,77.22,66,100,1.43,overcast clouds
39,39,Chuy,UY,-33.6971,-53.4616,76.03,57,64,9.17,broken clouds
40,40,Mar Del Plata,AR,-38.0023,-57.5575,80.62,55,0,13.00,clear sky


In [29]:
# 4a. Determine if there are any empty rows.
preferred_cities.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [30]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities.dropna()
clean_df.count()

City_ID                199
City                   199
Country                199
Lat                    199
Lng                    199
Max Temp               199
Humidity               199
Cloudiness             199
Wind Speed             199
Current Description    199
dtype: int64

In [31]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Port Elizabeth,ZA,75.09,clear sky,-33.9180,25.5701,
7,Bengkulu,ID,76.48,overcast clouds,-3.8004,102.2655,
13,Neiafu,TO,82.49,scattered clouds,-18.6500,-173.9833,
15,Hermanus,ZA,83.89,clear sky,-34.4187,19.2345,
16,Salinopolis,BR,75.54,overcast clouds,-0.6136,-47.3561,
20,Kavaratti,IN,81.19,clear sky,10.5669,72.6420,
21,Faanui,PF,80.33,moderate rain,-16.4833,-151.7500,
29,Ilebo,CD,77.22,overcast clouds,-4.3167,20.5833,
39,Chuy,UY,76.03,broken clouds,-33.6971,-53.4616,
40,Mar Del Plata,AR,80.62,clear sky,-38.0023,-57.5575,


In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")      

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [33]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Port Elizabeth,ZA,75.09,clear sky,-33.9180,25.5701,39 On Nile Guest House
7,Bengkulu,ID,76.48,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
13,Neiafu,TO,82.49,scattered clouds,-18.6500,-173.9833,Mystic Sands Beachfront Accommodation
15,Hermanus,ZA,83.89,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
16,Salinopolis,BR,75.54,overcast clouds,-0.6136,-47.3561,Salinas Park Resort Maçarico AP101
...,...,...,...,...,...,...,...
671,Roebourne,AU,84.43,broken clouds,-20.7833,117.1333,Latitude20 Roebourne Village
673,Sao Felix Do Xingu,BR,78.12,overcast clouds,-6.6447,-51.9950,Rio Xingu Camping
674,Mocuba,MZ,75.24,clear sky,-16.8392,36.9856,Phama Lodge
687,Pochutla,MX,86.83,clear sky,15.7432,-96.4661,Hotel Posada San Jose


In [34]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df[hotel_df['Hotel Name'] == '']
hotel_df = hotel_df[hotel_df["Hotel Name"] != '']
hotel_df.count()

City                   178
Country                178
Max Temp               178
Current Description    178
Lat                    178
Lng                    178
Hotel Name             178
dtype: int64

In [35]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Port Elizabeth,ZA,75.09,clear sky,-33.9180,25.5701,39 On Nile Guest House
7,Bengkulu,ID,76.48,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
13,Neiafu,TO,82.49,scattered clouds,-18.6500,-173.9833,Mystic Sands Beachfront Accommodation
15,Hermanus,ZA,83.89,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
16,Salinopolis,BR,75.54,overcast clouds,-0.6136,-47.3561,Salinas Park Resort Maçarico AP101
...,...,...,...,...,...,...,...
671,Roebourne,AU,84.43,broken clouds,-20.7833,117.1333,Latitude20 Roebourne Village
673,Sao Felix Do Xingu,BR,78.12,overcast clouds,-6.6447,-51.9950,Rio Xingu Camping
674,Mocuba,MZ,75.24,clear sky,-16.8392,36.9856,Phama Lodge
687,Pochutla,MX,86.83,clear sky,15.7432,-96.4661,Hotel Posada San Jose


In [36]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [37]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [40]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


In [41]:
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))